# 06 - Solução do Problema e Análise de Negócio

## Contextualização do Problema

Este notebook tem como objetivo responder às perguntas de negócio definidas na etapa inicial do MVP, utilizando os dados tratados e modelados na camada Gold (Data Warehouse).

A análise busca compreender como variáveis operacionais influenciam a ocorrência de falhas em equipamentos industriais, de forma a apoiar estratégias de manutenção preditiva.


## Perguntas de Negócio

1. Quais variáveis operacionais apresentam maior influência na ocorrência de falhas?
2. Existem padrões específicos de operação associados a falhas?
3. Como se comportam as principais métricas operacionais em situações de falha e operação normal?
4. Quais tipos de falha ocorrem com maior frequência?
5. Existem tipos de máquinas mais suscetíveis a falhas?


In [0]:
%sql
SELECT
  machine_failure,
  AVG(air_temperature_k) AS avg_air_temp,
  AVG(process_temperature_k) AS avg_process_temp,
  AVG(rotational_speed_rpm) AS avg_rpm,
  AVG(torque_nm) AS avg_torque,
  AVG(tool_wear_min) AS avg_tool_wear
FROM fact_maintenance
GROUP BY machine_failure;


machine_failure,avg_air_temp,avg_process_temp,avg_rpm,avg_torque,avg_tool_wear
0,299.9739985508732,309.9955698167888,1540.2600144912535,39.62965531518476,106.69371700652107
1,300.886430678466,310.2902654867255,1496.486725663717,50.168141592920364,143.78171091445427


A comparação entre operação normal e cenários de falha mostra que registros com falha apresentam, em média, **temperaturas mais elevadas**, **maior torque** e **maior desgaste da ferramenta**, enquanto operam com **menor velocidade de rotação**. Esse padrão indica maior estresse térmico e mecânico associado à ocorrência de falhas, reforçando a relevância dessas variáveis para estratégias de manutenção preditiva.


In [0]:
%sql
SELECT
  COUNT(*) AS total_registros,
  SUM(machine_failure) AS total_falhas,
  AVG(machine_failure) AS taxa_falha
FROM fact_maintenance;


total_registros,total_falhas,taxa_falha
10000,339,0.0339


A análise global indica uma taxa de falha de aproximadamente **3,39%**, com **339 falhas** em um total de **10.000 registros**. Esse resultado confirma que eventos de falha são relativamente raros no conjunto de dados, cenário típico em ambientes industriais, reforçando a necessidade de abordagens analíticas para sua identificação e prevenção.



In [0]:
%sql
SELECT
  machine_failure,
  AVG(rotational_speed_rpm) AS avg_rpm,
  AVG(torque_nm) AS avg_torque
FROM fact_maintenance
GROUP BY machine_failure;


machine_failure,avg_rpm,avg_torque
0,1540.2600144912535,39.62965531518476
1,1496.486725663717,50.168141592920364


A comparação entre os cenários de operação normal e falha evidencia que, nos registros com falha, as máquinas operam com **menor velocidade média de rotação** e **maior torque médio**, indicando maior esforço mecânico associado à ocorrência de falhas.



In [0]:
%sql
SELECT
  SUM(twf) AS tool_wear_failures,
  SUM(hdf) AS heat_dissipation_failures,
  SUM(pwf) AS power_failures,
  SUM(osf) AS overstrain_failures,
  SUM(rnf) AS random_failures
FROM fact_maintenance;


tool_wear_failures,heat_dissipation_failures,power_failures,overstrain_failures,random_failures
46,115,95,98,19


A distribuição dos tipos de falha mostra predominância de falhas relacionadas à **dissipação de calor (HDF)**, **sobrecarga (OSF)** e **potência (PWF)**, enquanto falhas por **desgaste da ferramenta (TWF)** e **eventos aleatórios (RNF)** ocorrem com menor frequência. Esse padrão indica que a maioria das falhas está associada a condições operacionais específicas e mensuráveis, reforçando a importância do monitoramento térmico, mecânico e elétrico para estratégias de manutenção preditiva.


In [0]:
%sql
SELECT
  d.type,
  AVG(f.machine_failure) AS taxa_falha_media
FROM fact_maintenance f
JOIN dim_machine d
  ON f.machine_sk = d.machine_sk
GROUP BY d.type
ORDER BY taxa_falha_media DESC;


type,taxa_falha_media
L,0.03916666666666667
M,0.02769436102769436
H,0.020937188434695914


A análise da taxa média de falha por tipo de máquina evidencia que equipamentos do tipo **L** apresentam maior propensão a falhas, seguidos pelos tipos **M**, enquanto máquinas do tipo **H** apresentam a menor taxa de falha. Esse resultado indica comportamentos operacionais distintos entre os grupos, reforçando a necessidade de estratégias de manutenção diferenciadas por tipo de equipamento.



## Discussão Geral da Solução do Problema

De forma geral, as análises indicam que as falhas ocorrem em um contexto de maior estresse operacional, caracterizado por temperaturas mais elevadas, aumento do torque e maior desgaste da ferramenta, frequentemente acompanhado por redução da velocidade de rotação. Apesar de a taxa global de falha ser relativamente baixa (aproximadamente 3,4%), observa-se variação relevante entre os tipos de máquina, com maior incidência em determinados grupos. Além disso, a predominância de falhas relacionadas à dissipação de calor, sobrecarga e potência reforça a influência direta das condições operacionais no desempenho dos equipamentos. Esses resultados demonstram que a modelagem dimensional e o pipeline implementados são adequados para identificar padrões críticos de falha, fornecendo uma base sólida para estratégias de manutenção preditiva e suporte à tomada de decisão baseada em dados.
